**ReAct agent from scratch**

In [28]:
# Importing required libraries
import os
import re
import httpx
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
# Loading the environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [19]:
# Loading the LLM model
llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0)
# llm.invoke("HI").content

message=[
    {"role":"system","content":"you are a helpful assistant"},
    {"role":"user","content":"hi how are you?"}
    ]

result = llm.invoke(message)
print(result.content)

Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always happy to help and assist with any questions or tasks you may have. How about you? How's your day going so far? Is there anything I can help you with?


In [14]:
# Defining the class to create Chatbot
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message = []
        if self.system:
            self.message.append({"role":"system","content":system})
    
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result = self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
    
    def execute(self):
        llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0)
        result = llm.invoke(self.message)
        return result.content
    
# calling object
bot = Chatbot("you are a helful assistant")
bot("Hi")
bot("Do you know Rohit Sharma ?")
bot.message

[{'role': 'system', 'content': 'you are a helful assistant'},
 {'role': 'user', 'content': 'Hi'},
 {'role': 'assistant',
  'content': "It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss."},
 {'role': 'user', 'content': 'Do you know Rohit Sharma ?'},
 {'role': 'assistant',
  'content': 'Rohit Sharma is a well-known Indian international cricketer. He is a right-handed batsman and is considered one of the most successful opening batsmen in the history of the Indian cricket team.\n\nRohit has had a highly successful career, holding numerous records in international cricket, including the record for the highest individual score in an ODI (One Day International) match, which is 264 runs. He has also led the Mumbai Indians to multiple Indian Premier League (IPL) titles and has been a key player for the Indian team in various international tournaments.\n\nRohit is known for his elega

In [33]:
# Define prompt
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [27]:
# regular expression to identify Action to take
action_re = re.compile('^Action: (\w+): (.*)')

prompt1 = "Action: Move: Forward"
prompt2 = "Action: call: Calculate"
prompt3 = "Hrishikesh Kothawade"

match = action_re.match(prompt3)
print(match)

None


In [31]:
# create tool functions
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]


def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]


def calculate(number):
    return eval(number)

In [32]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [35]:
bot = Chatbot(prompt)
next_prompt="Tell me about quantum computing from Wikipedia."
result = bot(next_prompt)
result

'Thought: To provide information about quantum computing, I should search for the term "Quantum computing" on Wikipedia to get a summary of the topic.\n\nAction: wikipedia: Quantum computing\nPAUSE'

In [41]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
actions[0].groups()

('wikipedia', 'Quantum computing')

In [49]:
# We will ask the query to function, which will perform further things for us
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            # print(bot.message)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [50]:
query("2*4*5/8+9*789-451")

Thought: To solve this mathematical expression, I need to follow the order of operations (PEMDAS) and calculate the result step by step. 

Action: calculate: 2*4*5/8 + 9*789 - 451
PAUSE
 -- running calculate 2*4*5/8 + 9*789 - 451
Observation: 6655.0
Thought: The calculation has been performed, and the result is available.

Answer: 6655.0


'Thought: The calculation has been performed, and the result is available.\n\nAnswer: 6655.0'